In [2]:
import warnings
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
warnings.simplefilter("ignore")
%matplotlib inline

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [2]:
#!pip install tqdm


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\python.exe -m pip install --upgrade pip' command.


In [3]:
data = pd.read_csv("./data/assignment_2_train.csv")


In [8]:
data_ldb = pd.read_csv("./data/assignment_2_test.csv")

In [1]:
data_ldb.head(10)

NameError: name 'data_ldb' is not defined

In [8]:
x_train, x_valid = train_test_split(
    data.drop(["TransactionID", "isFraud"], axis=1), train_size=0.75, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    data["isFraud"], train_size=0.75, shuffle=True, random_state=1,
)

In [9]:
x_ldb = data_ldb.drop(["TransactionID", "isFraud"], axis=1)
y_ldb = data_ldb["isFraud"]

In [24]:
y_ldb.head(10)

0    1
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: isFraud, dtype: int64

In [26]:
numerical_features = x_train.select_dtypes(exclude=["object"])
numerical_features = numerical_features.columns.tolist()

x_train = x_train[numerical_features]
x_valid = x_valid[numerical_features]
x_ldb = x_ldb[numerical_features]

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("x_ldb.shape = {} rows, {} cols".format(*x_ldb.shape))

x_train.shape = 135000 rows, 378 cols
x_valid.shape = 45000 rows, 378 cols
x_ldb.shape = 100001 rows, 378 cols


In [47]:
model = xgb.XGBClassifier(n_estimators=10000,random_state=1)
eval_set = [(x_valid, y_valid)]
model.fit(x_train, y_train,early_stopping_rounds=10, eval_metric="auc", eval_set=eval_set, verbose=True)

[0]	validation_0-auc:0.75005
[1]	validation_0-auc:0.81958
[2]	validation_0-auc:0.82705
[3]	validation_0-auc:0.82888
[4]	validation_0-auc:0.83463
[5]	validation_0-auc:0.84026
[6]	validation_0-auc:0.84202
[7]	validation_0-auc:0.85946
[8]	validation_0-auc:0.86192
[9]	validation_0-auc:0.87803
[10]	validation_0-auc:0.88116
[11]	validation_0-auc:0.88426
[12]	validation_0-auc:0.88842
[13]	validation_0-auc:0.89227
[14]	validation_0-auc:0.89477
[15]	validation_0-auc:0.89906
[16]	validation_0-auc:0.90034
[17]	validation_0-auc:0.90242
[18]	validation_0-auc:0.90376
[19]	validation_0-auc:0.90503
[20]	validation_0-auc:0.90779
[21]	validation_0-auc:0.90908
[22]	validation_0-auc:0.90973
[23]	validation_0-auc:0.90967
[24]	validation_0-auc:0.91059
[25]	validation_0-auc:0.91146
[26]	validation_0-auc:0.91316
[27]	validation_0-auc:0.91408
[28]	validation_0-auc:0.91436
[29]	validation_0-auc:0.91505
[30]	validation_0-auc:0.91517
[31]	validation_0-auc:0.91554
[32]	validation_0-auc:0.91646
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=10000, n_jobs=4, num_parallel_tree=1,
       objective='binary:logistic', random_state=1, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       use_label_encoder=True, validate_parameters=1, verbosity=None)

In [48]:
train_roc_score = roc_auc_score(y_train, model.predict(x_train))
valid_roc_score = roc_auc_score(y_valid, model.predict(x_valid))
ldb_roc_score = roc_auc_score(y_ldb, model.predict(x_ldb))

print(f"Train-ROC_score: {round(train_roc_score, 3)}, Valid-ROC_score: {round(valid_roc_score, 3)}, LDB-ROC_score: {round(ldb_roc_score, 3)}")

Train-ROC_score: 0.885, Valid-ROC_score: 0.771, LDB-ROC_score: 0.647


In [ ]:
# Вывод модель сильно переобучена.

## Задание 2

сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [49]:
x_train, x_valid = train_test_split(
    data.drop(["TransactionID", "isFraud"], axis=1), train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    data["isFraud"], train_size=0.7, shuffle=True, random_state=1,
)


In [50]:
x_train = x_train[numerical_features]
x_valid = x_valid[numerical_features]

In [51]:
x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [52]:
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 125999 rows, 378 cols
x_valid.shape = 37800 rows, 378 cols
x_test.shape = 16201 rows, 378 cols


In [57]:
model = xgb.XGBClassifier(n_estimators=10000,random_state=1)

In [58]:
eval_set = [(x_valid, y_valid)]
model.fit(x_train, y_train,early_stopping_rounds=10, eval_metric="auc", eval_set=eval_set, verbose=True)

[0]	validation_0-auc:0.74280
[1]	validation_0-auc:0.79520
[2]	validation_0-auc:0.82118
[3]	validation_0-auc:0.82388
[4]	validation_0-auc:0.82799
[5]	validation_0-auc:0.85819
[6]	validation_0-auc:0.85996
[7]	validation_0-auc:0.86458
[8]	validation_0-auc:0.86806
[9]	validation_0-auc:0.87661
[10]	validation_0-auc:0.88228
[11]	validation_0-auc:0.88376
[12]	validation_0-auc:0.88700
[13]	validation_0-auc:0.89157
[14]	validation_0-auc:0.89548
[15]	validation_0-auc:0.89734
[16]	validation_0-auc:0.90186
[17]	validation_0-auc:0.90310
[18]	validation_0-auc:0.90497
[19]	validation_0-auc:0.90655
[20]	validation_0-auc:0.90802
[21]	validation_0-auc:0.90870
[22]	validation_0-auc:0.90952
[23]	validation_0-auc:0.91119
[24]	validation_0-auc:0.91144
[25]	validation_0-auc:0.91313
[26]	validation_0-auc:0.91461
[27]	validation_0-auc:0.91503
[28]	validation_0-auc:0.91604
[29]	validation_0-auc:0.91681
[30]	validation_0-auc:0.91849
[31]	validation_0-auc:0.91930
[32]	validation_0-auc:0.92003
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=130, n_jobs=4, num_parallel_tree=1,
       objective='binary:logistic', random_state=1, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       use_label_encoder=True, validate_parameters=1, verbosity=None)

In [55]:
train_roc_score = roc_auc_score(y_train, model.predict(x_train))
valid_roc_score = roc_auc_score(y_valid, model.predict(x_valid))
test_roc_score = roc_auc_score(y_test, model.predict(x_test))
ldb_roc_score = roc_auc_score(y_ldb, model.predict(x_ldb))


In [56]:
print(f"Train-ROC_score: {round(train_roc_score, 3)}, Valid-ROC_score: {round(valid_roc_score, 3)},Test-ROC_score: {round(test_roc_score, 3)}, LDB-ROC_score: {round(ldb_roc_score, 3)}")

Train-ROC_score: 0.847, Valid-ROC_score: 0.761,Test-ROC_score: 0.762, LDB-ROC_score: 0.666


In [ ]:
#На тестовой и валидационной выборке видно, что модель переобучена. Плохой результат на лидерборде.

## Задание 3

Задание 3: построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

In [59]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


In [60]:
def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores

In [61]:
def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [71]:
scores_train = create_bootstrap_metrics(y_train, model.predict(x_train), roc_auc_score)
scores_valid = create_bootstrap_metrics(y_valid, model.predict(x_valid), roc_auc_score)
scores_test = create_bootstrap_metrics(y_test, model.predict(x_test), roc_auc_score)
#scores_ldb = create_bootstrap_metrics(y_ldb, model.predict(x_ldb), roc_auc_score)

In [75]:
print(f"Доверительный интеравал для train: {calculate_confidence_interval(scores_train)}")
print(f"Доверительный интеравал для valid: {calculate_confidence_interval(scores_valid)}")
print(f"Доверительный интеравал для test: {calculate_confidence_interval(scores_test)}")
#print(f"Доверительный интеравал для ldb: {calculate_confidence_interval(scores_ldb)}")
print(f"LDB-ROC_score: {round(ldb_roc_score, 3)}")

Доверительный интеравал для train: (0.839673124714728, 0.8549501759221684)
Доверительный интеравал для valid: (0.7462812726251457, 0.7773021208253119)
Доверительный интеравал для test: (0.7402114849463521, 0.7830587446381373)
LDB-ROC_score: 0.666


In [ ]:
#Качество на лидерборде вне доверительного интервала, валидация плохая

## Задание 4

Задание 4: выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из
assignment_2_test.csv
, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

In [6]:
x_train=data.drop(["TransactionID", "isFraud"], axis=1)
y_train=data["isFraud"]

In [10]:
x_adv = pd.concat([
    x_train, x_ldb], axis=0
)
y_adv = np.hstack((np.zeros(x_train.shape[0]), np.ones(x_ldb.shape[0])))
assert x_adv.shape[0] == y_adv.shape[0]

In [12]:
numerical_features = x_train.select_dtypes(exclude=["object"])
numerical_features = numerical_features.columns.tolist()

In [13]:
x_adv = x_adv[numerical_features]

In [14]:
x_train=x_train[numerical_features]

In [15]:
model = xgb.XGBClassifier(n_estimators=10000,random_state=1)

In [16]:
eval_set = [(x_adv, y_adv)]
model.fit(x_train, y_train,early_stopping_rounds=10, eval_metric="auc", eval_set=eval_set, verbose=True)

[0]	validation_0-auc:0.49362
[1]	validation_0-auc:0.49485
[2]	validation_0-auc:0.49368
[3]	validation_0-auc:0.49468
[4]	validation_0-auc:0.49505
[5]	validation_0-auc:0.49700
[6]	validation_0-auc:0.52729
[7]	validation_0-auc:0.52744
[8]	validation_0-auc:0.52615
[9]	validation_0-auc:0.52196
[10]	validation_0-auc:0.53690
[11]	validation_0-auc:0.54694
[12]	validation_0-auc:0.55363
[13]	validation_0-auc:0.54109
[14]	validation_0-auc:0.54908
[15]	validation_0-auc:0.55932
[16]	validation_0-auc:0.55535
[17]	validation_0-auc:0.55405
[18]	validation_0-auc:0.55676
[19]	validation_0-auc:0.55532
[20]	validation_0-auc:0.58495
[21]	validation_0-auc:0.58218
[22]	validation_0-auc:0.58268
[23]	validation_0-auc:0.58311
[24]	validation_0-auc:0.58189
[25]	validation_0-auc:0.58976
[26]	validation_0-auc:0.58994
[27]	validation_0-auc:0.58975
[28]	validation_0-auc:0.59743
[29]	validation_0-auc:0.59255
[30]	validation_0-auc:0.59324
[31]	validation_0-auc:0.59527
[32]	validation_0-auc:0.59568
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=10000, n_jobs=4, num_parallel_tree=1,
       objective='binary:logistic', random_state=1, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       use_label_encoder=True, validate_parameters=1, verbosity=None)

In [18]:
x_ldb = x_ldb[numerical_features]


In [19]:
train_roc_score = roc_auc_score(y_train, model.predict(x_train))
ldb_roc_score = roc_auc_score(y_ldb, model.predict(x_ldb))


In [22]:
print(f"Train-ROC_score: {round(train_roc_score, 3)}, LDB-ROC_score: {round(ldb_roc_score, 3)}")

Train-ROC_score: 0.751, LDB-ROC_score: 0.654


## Задание 5

Задание 5: сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

In [23]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train)
        y_train_pred = estimator.predict(x_train)
        y_valid_pred = estimator.predict(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [ ]:
cv_strategy = KFold(n_splits=5, random_state=1)

estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    data[numerical_features], data["isFraud"], model, metric=roc_auc_score, cv_strategy=cv_strategy
)

[21:58:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
